<div style='background-color: #fff0db; border: 3px solid #FFA07A; border-radius: 10px; padding: 6px; text-align: center;'>
<font size="+1.8" color="#FF4500"><b>🍎 1. IMPORT LIBRARY 🍎
</b></font>
</div>

In [1]:
import os
import numpy as np
from glob import glob
from PIL import Image
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

np.random.seed(2025)

ModuleNotFoundError: No module named 'sklearn'

<div style='background-color: #fff0db; border: 3px solid #FFA07A; border-radius: 10px; padding: 6px; text-align: center;'>
<font size="+1.8" color="#FF4500"><b>🍎 2. DATA LOADER 🍎</b></font>
</div>

In [ ]:
class CustomDataset:
    def __init__(self, root, data_type, transformations=None, im_files=[".png", ".jpg", ".jpeg"]):
        self.transformations, self.data_type = transformations, data_type
        self.im_paths = []
        for im_file in im_files:
            self.im_paths.extend(glob(f"{root}/{data_type}/*/*{im_file}"))

        self.cls_names, self.cls_counts = {}, {}
        count = 0
        for im_path in self.im_paths:
            class_name = self.get_class(im_path)
            if class_name not in self.cls_names:
                self.cls_names[class_name] = count
                self.cls_counts[class_name] = 1
                count += 1
            else:
                self.cls_counts[class_name] += 1        

    def get_class(self, path):
        return os.path.dirname(path).split("/")[-1]

    def __len__(self):
        return len(self.im_paths)

    def __getitem__(self, idx):
        im_path = self.im_paths[idx]
        im = Image.open(im_path).convert("RGB")
        if self.transformations:
            im = self.transformations(im)
        gt = self.cls_names[self.get_class(im_path)]
        return im, gt

    def get_data(self):
        images, labels = [], []
        for idx in range(len(self.im_paths)):
            im, label = self[idx]
            images.append(im)
            labels.append(label)
        return np.array(images), np.array(labels)

<div style='background-color: #fff0db; border: 3px solid #FFA07A; border-radius: 10px; padding: 6px; text-align: center;'>
<font size="+1.8" color="#FF4500"><b>🍎 3. PREPROCESSING & LOAD DATASET 🍎</b></font>
</div>

In [ ]:
def preprocess(image, im_size=64):
    image = image.resize((im_size, im_size))
    image = np.array(image).astype(np.float32) / 255.0
    return image

root = "/kaggle/input/fruit-ripeness-unripe-ripe-and-rotten/archive (1)/dataset/dataset"
im_size = 64

tfs = lambda im: preprocess(im, im_size=im_size)

train_dataset = CustomDataset(root=root, data_type="train", transformations=tfs)
test_dataset = CustomDataset(root=root, data_type="test", transformations=tfs)

X_train_full, y_train_full = train_dataset.get_data()
X_test, y_test = test_dataset.get_data()

# Split Train-Validation
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=2025, stratify=y_train_full)

print(f"Train Shape: {X_train.shape}, Validation Shape: {X_val.shape}, Test Shape: {X_test.shape}")
print(f"Jumlah Data:")
print(f"- Train      : {len(X_train)} samples")
print(f"- Validation : {len(X_val)} samples")
print(f"- Test       : {len(X_test)} samples")

<div style='background-color: #fff0db; border: 3px solid #FFA07A; border-radius: 10px; padding: 6px; text-align: center;'>
<font size="+1.8" color="#FF4500"><b>🍎 4. VISUALISASI DATASET🍎</b></font>
</div>

In [ ]:
class_names = list(train_dataset.cls_names.keys())

plt.figure(figsize=(12, 6))
for i in range(6):
    idx = random.randint(0, len(X_train) - 1)
    plt.subplot(2, 3, i + 1)
    plt.imshow(X_train[idx])
    plt.title(f"Label: {class_names[y_train[idx]]}")
    plt.axis('off')
plt.show()

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

<div style='background-color: #fff0db; border: 3px solid #FFA07A; border-radius: 10px; padding: 6px; text-align: center;'>
<font size="+1.8" color="#FF4500"><b>🍎 5. MODEL CNN🍎</b></font>
</div>

In [ ]:
num_classes = len(train_dataset.cls_names)

cnn = models.Sequential([
    layers.Input(shape=(im_size, im_size, 3)),
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3), 
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  
    layers.Dense(num_classes, activation='softmax')
])


cnn.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

cnn.summary()

<div style='background-color: #fff0db; border: 3px solid #FFA07A; border-radius: 10px; padding: 6px; text-align: center;'>
<font size="+1.8" color="#FF4500"><b>🍎 6. CALLBACK 🍎</b></font>
</div>

In [ ]:
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

<div style='background-color: #fff0db; border: 3px solid #FFA07A; border-radius: 10px; padding: 6px; text-align: center;'>
<font size="+1.8" color="#FF4500"><b>🍎 7. TRAINING CNN 🍎</b></font>
</div>

In [ ]:
history = cnn.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=1
)

<div style='background-color: #fff0db; border: 3px solid #FFA07A; border-radius: 10px; padding: 6px; text-align: center;'>
<font size="+1.8" color="#FF4500"><b>🍎 8. VISUALISASI TRAINING 🍎</b></font>
</div>

In [ ]:
plt.figure(figsize=(14, 5))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()

plt.show()

<div style='background-color: #fff0db; border: 3px solid #FFA07A; border-radius: 10px; padding: 6px; text-align: center;'>
<font size="+1.8" color="#FF4500"><b>🍎 9. EVALUASI MODEL 🍎</b></font>
</div>

In [ ]:
best_model = tf.keras.models.load_model('best_model.keras')
test_loss, test_acc = best_model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_acc:.4f}")

<div style='background-color: #fff0db; border: 3px solid #FFA07A; border-radius: 10px; padding: 6px; text-align: center;'>
<font size="+1.8" color="#FF4500"><b>🍎 10. METRIK & CONFUSION MATRIX 🍎</b></font>
</div>

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Prediksi pada data test
y_pred = np.argmax(best_model.predict(X_test), axis=1)

# Classification Report
report = classification_report(y_test, y_pred, target_names=class_names, digits=4)
print(report)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.title('Confusion Matrix')
plt.show()

<div style='background-color: #fff0db; border: 3px solid #FFA07A; border-radius: 10px; padding: 6px; text-align: center;'>
<font size="+1.8" color="#FF4500"><b>🍎 11. VISUALISASI PREDIKSI 🍎</b></font>
</div>

In [ ]:
plt.figure(figsize=(15, 5))
indices = random.sample(range(len(X_test)), 5)

for i, idx in enumerate(indices):
    img = X_test[idx]
    label = y_test[idx]
    prediction = np.argmax(best_model.predict(np.expand_dims(img, axis=0)))
    
    plt.subplot(1, 5, i + 1)
    plt.imshow(img)
    plt.title(f"True: {class_names[label]}\nPred: {class_names[prediction]}")
    plt.axis('off')

plt.show()

<div style='background-color: #fff0db; border: 3px solid #FFA07A; border-radius: 10px; padding: 6px; text-align: center;'>
<font size="+1.8" color="#FF4500"><b>🍎 12. CONFIDENCE 🍎</b></font>
</div>

In [ ]:
idx = np.random.choice(len(X_test), 9, replace=False)
y_pred = cnn.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
plt.figure(figsize=(12, 12))
for i, index in enumerate(idx):
    plt.subplot(3, 3, i + 1)
    plt.imshow(X_test[index])
    true_label = list(train_dataset.cls_names.keys())[list(train_dataset.cls_names.values()).index(y_test[index])]
    pred_label = list(train_dataset.cls_names.keys())[y_pred_labels[index]]
    confidence = np.max(y_pred[index]) * 100  # Confidence score dalam persen
    plt.title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.2f}%')
    plt.axis('off')
plt.show()

<div style='background-color: #fff0db; border: 3px solid #FFA07A; border-radius: 10px; padding: 6px; text-align: center;'>
<font size="+1.8" color="#FF4500"><b>🍎 13. KONVERSI KE TFLITE 🍎</b></font>
</div>

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
tflite_model = converter.convert()

# Simpan TFLite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model berhasil disimpan sebagai model.tflite")